In [ ]:
# we load the things!

from ngsolve import *
from ngsolve.meshes import MakeStructured3DMesh
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.geom2d import SplineGeometry
from ngsolve.solvers import GMRes

In [ ]:

def createReentrantCornerGeometry(hmax):

    largeBrick = Box(Pnt(-0.5, -0.5,-0.5), Pnt(0.5, 0.5, 0.5))
    smallBrick = Box(Pnt(-0.5, -0.5,-0.5), Pnt(0, 0, 0))

    reentrantCornerGeo3D = largeBrick - smallBrick
    
    reentrantCornerGeo3D.faces.Min(X).name = "minX"
    reentrantCornerGeo3D.faces.Max(X).name = "maxX"
    reentrantCornerGeo3D.faces.Min(Y).name = "minY"
    reentrantCornerGeo3D.faces.Max(Y).name = "maxY"
    reentrantCornerGeo3D.faces.Min(Z).name = "minZ"
    reentrantCornerGeo3D.faces.Max(Z).name = "maxZ"

    mesh = Mesh(OCCGeometry(reentrantCornerGeo3D).GenerateMesh(maxh=hmax))

    return mesh

In [ ]:
# Hodge Laplace for 1-forms

useGMRes = True

def hodgeLaplace1Forms(mesh,
                       f = CF((0,0,0)), # this is the right hand side f
                       order = 1,
                       C_w = 1,
                       dirichletBnd = None,
                       gOnDirichletBnd = None,
                       ):
    
    h_curl = HCurl(mesh, order=order, type1=True)  # For 1-forms, H(curl)
    h_1 = H1(mesh, order=order)     # For 0-forms, H1 space
    fes = h_1 * h_curl
    (p, u), (q, v) = fes.TnT()

    n = specialcf.normal(mesh.dim)
    h = specialcf.mesh_size

    B, F = BilinearForm(fes), LinearForm(fes)

    B += u * grad(q) * dx
    B += - p * q * dx
    
    B +=  grad(p) * v * dx
    B +=  curl(u) * curl(v) * dx

    F += f * v * dx
    
    for boundary_name, boundary_g in zip(dirichletBnd, gOnDirichletBnd):
        dS_dirichlet = ds(skeleton=True, definedon=mesh.Boundaries(boundary_name))
        
        B += - curl(u) * Cross(n, v) * dS_dirichlet
        B += - Cross(n, u) * curl(v) * dS_dirichlet
        B += (C_w / h) * Cross(n, u) * Cross(n, v) * dS_dirichlet
            
        F += (C_w / h) * Cross(n, boundary_g) * Cross(n, v) * dS_dirichlet
        F += - Cross(n, boundary_g) * curl(v) * dS_dirichlet
        F += boundary_g * n * q * dS_dirichlet

    with TaskManager(): 
        if (useGMRes == False):
            B.Assemble()
            F.Assemble()
            sol = GridFunction(fes)
            res = F.vec-B.mat * sol.vec
            inv = B.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
            sol.vec.data += inv * res
        else:
            B.Assemble()
            F.Assemble()
            sol = GridFunction(fes)
            blocks = fes.CreateSmoothingBlocks()
            prebj = B.mat.CreateBlockSmoother(blocks)
            GMRes(A =B.mat,x= sol.vec, b=F.vec,pre = prebj,  printrates="\r", maxsteps = 10000, tol=1e-8)
    gf_p , gf_u = sol.components

    return gf_u, gf_p

In [ ]:

mesh = createReentrantCornerGeometry(0.1)
C_w = 1000
order = 2

n = specialcf.normal(mesh.dim)
t = specialcf.tangential(mesh.dim)

zero = CF((0,0,0))

bndList = ['minX', 'minY', 'minZ',  'maxY', 'maxX', 'maxZ', 'default']
gList = [-0.1*n, 0.1*n, 0.1*n, zero, zero, zero, zero]

f = 1 * CF((3, -3, -3))

gfu, gfp = hodgeLaplace1Forms(mesh, f, order, C_w, bndList, gList)
clipping ={"pnt":(0,0,-0.01), "function":True, "vec":(0,0,-1)}
Draw(gfu, mesh, clipping=clipping)
